# [Module 1] 데이타 준비 (Prepare data file)

* 이 노트북에서는 입력 데이타를 Forecast에 입력이 될 포맷으로 학습 (Train) 과 검증셋(Validation)으로 나누는 작업을 합니다. **자세한 과정을 알고 싶으시면 0.LookAt_RawData 노트북 을 실행 해주세요.** (Please refer to the 0.LookAt_RawData.ipynb if you see a process of handling a raw data file handled)
    - 데이타는 아래 데이타 소스에서 train.csv 파일을 다운로드 받아서 data 폴더에 train.csv로 저장한 것을 사용 합니다.
    - Data Source: Store Item Demand Forecasting Challenge,  https://www.kaggle.com/c/demand-forecasting-kernels-only/overview
* 이 노트북은 약 3분 정도 소요 됩니다. **About 3 mins may be elapsed**

In [12]:
import boto3
from time import sleep
import os
import pandas as pd
import json
import time
import pprint
import numpy as np

## Create Target Time Series File and Validation File
- 실제 Raw 데이터를 Forecast에 들어갈 데이타 포맷으로 바꾸는 작업을 합니다.

In [23]:
data_dir = 'data'
train_file_name = 'train.csv'
train_data = pd.read_csv(os.path.join(data_dir,train_file_name))
train_df = train_data.copy()
train_df = train_df.rename(columns={'item':'item_id'})

In [24]:
train_df = train_df.set_index('date')
train_df.store = train_df.store.astype(str)
train_df.item_id = train_df.item_id.astype(str)
train_df.index = pd.to_datetime(train_df.index, format = '%Y-%m-%d')
# train_df.index = train_df.index.strftime('%Y-%m-%d %H:%M:%S')
cols_order = ['item_id', 'store','sales']
train_df = train_df[cols_order]

train_df.head()

item_id store  sales
date                           
2013-01-01       1     1     13
2013-01-02       1     1     11
2013-01-03       1     1     14
2013-01-04       1     1     13
2013-01-05       1     1     10

In [25]:
train_df.tail()

item_id store  sales
date                           
2017-12-27      50    10     63
2017-12-28      50    10     59
2017-12-29      50    10     74
2017-12-30      50    10     62
2017-12-31      50    10     82

## Split into target file and validation file
- 실제 데이타는 5년의 데이타가 있지만, 학습 속도를 빠르게 하기 위해서 2년치의 데이타만을 사용합니다. 2년 데이타에서 23개월은 학습 데이터로 사용을 하고 마지막 1개월은 검증 데이터로 사용 합니다.

In [26]:
print(train_df.index.max())
print(train_df.index.min())
start_train_date = '2015-01-01' # Inclusive
end_train_date = '2017-12-01' # Inclusive
end_val_date = '2018-01-01' # Non-inclusive
stores_sales = train_df.copy()

2017-12-31 00:00:00
2013-01-01 00:00:00


In [28]:
total_stores_sales=stores_sales[stores_sales.index >= start_train_date]
total_stores_sales=total_stores_sales[total_stores_sales.index< end_val_date]

total_target_filename = "total_target_time_series.csv"
total_target_path = data_dir + "/" + total_target_filename
total_stores_sales.to_csv(total_target_path, header=False)

In [17]:
target_stores_sales = stores_sales[stores_sales.index >= start_train_date]
target_stores_sales = target_stores_sales[target_stores_sales.index < end_train_date]
validation_stores_sales = stores_sales[stores_sales.index >= end_train_date]
validation_stores_sales = validation_stores_sales[validation_stores_sales.index < end_val_date]


In [18]:
target_stores_sales.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 532500 entries, 2015-01-01 to 2017-11-30
Data columns (total 3 columns):
item_id    532500 non-null object
store      532500 non-null object
sales      532500 non-null int64
dtypes: int64(1), object(2)
memory usage: 16.3+ MB


In [19]:
validation_stores_sales.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15500 entries, 2017-12-01 to 2017-12-31
Data columns (total 3 columns):
item_id    15500 non-null object
store      15500 non-null object
sales      15500 non-null int64
dtypes: int64(1), object(2)
memory usage: 484.4+ KB


- CSV 파일에 학습과 검증셋의 데이타를 저장 합니다.

In [20]:
# With the data in a great state, save it off as a CSV
target_time_series_filename = "target_time_series.csv"
target_time_series_path = data_dir + "/" + target_time_series_filename
target_stores_sales.to_csv(target_time_series_path, header=False)

In [21]:
# With the data in a great state, save it off as a CSV
validation_time_series_filename = "validation_time_series.csv"
validation_time_series_path = data_dir + "/" + validation_time_series_filename
validation_stores_sales.to_csv(validation_time_series_path, header=False)

- 다음 노트북에서 사용을 하기 위해서, 이 노트북에서 저장한 변수를 디스크에 저장을 합니다.
만약 아래의 코드에 에러가 발생을 했다고 하면, 이 노트북을 처음부터 다시 실행 해주세요.

In [29]:
%store target_time_series_filename
%store target_time_series_path
%store target_stores_sales

%store validation_time_series_filename
%store validation_time_series_path
%store validation_stores_sales

%store total_target_filename
%store total_target_path 
%store total_stores_sales

%store start_train_date 
%store end_train_date 
%store end_val_date

Stored 'target_time_series_filename' (str)
Stored 'target_time_series_path' (str)
Stored 'target_stores_sales' (DataFrame)
Stored 'validation_time_series_filename' (str)
Stored 'validation_time_series_path' (str)
Stored 'validation_stores_sales' (DataFrame)
Stored 'total_target_filename' (str)
Stored 'total_target_path' (str)
Stored 'total_stores_sales' (DataFrame)
Stored 'start_train_date' (str)
Stored 'end_train_date' (str)
Stored 'end_val_date' (str)
